##### Copyright 2020 The TensorFlow IO Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Avro 데이터세트 API

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/io/tutorials/avro"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> TensorFlow.org에서 보기</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/io/tutorials/avro.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/io/tutorials/avro.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서소스 보기</a></td>
      <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/io/tutorials/avro.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운론드하기</a></td>
</table>

## 개요

Avro 데이터세트 API의 목적은 Avro 형식의 데이터를 <a target="_blank" href="https://www.tensorflow.org/api_docs/python/tf/data/Dataset">TensorFlow 데이터세트</a>로 TensorFlow에 기본적으로 로드하는 것입니다. Avro는 프로토콜 버퍼와 유사한 데이터 직렬화 시스템입니다. 영구 데이터를 위한 직렬화 형식과 Hadoop 노드 간의 통신을 위한 와이어 형식을 모두 제공할 수 있는 Apache Hadoop에서 널리 사용됩니다. Avro 데이터는 행 지향의 압축된 바이너리 데이터 형식으로, 별도의 JSON 파일로 저장된 스키마에 의존합니다. Avro 형식 및 스키마 선언의 사양은 <a target="_blank" href="https://avro.apache.org/docs/current/spec.html">공식 매뉴얼</a>을 참조하세요.


## 설치 패키지


### 필요한 tensorflow-io 패키지 설치하기

In [ ]:
!pip install tensorflow-io

### 패키지 가져오기

In [ ]:
model.fit(x=dataset, epochs=1, steps_per_epoch=1, verbose=1)


### tf 및 tfio 가져오기 검증하기

In [ ]:
print("tensorflow-io version: {}".format(tfio.__version__))
print("tensorflow version: {}".format(tf.__version__))

## 사용법

### 데이터세트 살펴보기

이 튜토리얼의 목적을 위해 샘플 Avro 데이터세트를 다운로드하겠습니다.


샘플 Avro 파일 다운로드:

In [ ]:
!curl -OL https://github.com/tensorflow/io/raw/master/docs/tutorials/avro/train.avro
!ls -l train.avro

샘플 Avro 파일의 해당 스키마 파일을 다운로드합니다.

In [ ]:
!curl -OL https://github.com/tensorflow/io/raw/master/docs/tutorials/avro/train.avsc
!ls -l train.avsc

위의 예에서 테스트용 Avro 데이터세트는 mnist 데이터세트를 기반으로 생성되었습니다. TFRecord 형식의 원래 mnist 데이터세트는 <a target="_blank" href="https://www.tensorflow.org/datasets/api_docs/python/tfds/load">TF라는 데이터세트</a>에서 생성됩니다. 그러나 mnist 데이터세트는 데모 데이터세트로 사용하기엔 너무 큽니다. 단순화하기 위해 대부분을 잘라내고 처음 몇 개의 레코드만 보관했습니다. 또한 원래 mnist 데이터세트의 `image` 필드에 대해 추가 잘라내기가 수행되고 Avro의 `features` 필드에 매핑되었습니다. 따라서 `dataType` 파일 `train.avro`에는 4개의 레코드가 있으며, 그 각각에는 int 배열인 `features`, int 또는 null인 `label`, enum인 `dataType` 등 세 가지 필드가 있습니다. 디코딩된 `train.avro`를 보려면 다음과 같이 합니다(avro는 압축된 형식이므로 <a target="_blank" href="https://github.com/tensorflow/io/raw/master/docs/tutorials/avro/train.avro">원본 avro 데이터 파일</a>은 사람이 읽을 수 없음).


Avro 파일을 읽는 데 필요한 패키지를 설치합니다.


In [ ]:
!pip install avro


사람이 읽을 수 있는 형식으로 Avro 파일을 읽고 인쇄하는 방법:


In [ ]:
dataset = tfio.experimental.columnar.make_avro_record_dataset(file_pattern=['train.avro'],
                                                              reader_schema=schema,
                                                              features=features,
                                                              shuffle=False,
                                                              num_parallel_reads=16,
                                                              batch_size=3,
                                                              drop_final_batch=True,
                                                              num_epochs=1)

for record in dataset:
    print(record)


그리고 `train.avro`가 나타내는 `train.avsc`의 스키마는 JSON 형식 파일입니다. `train.avsc`를 보려면 다음과 같이 합니다.


In [ ]:
def print_schema(avro_schema_file):
    with open(avro_schema_file, 'r') as handle:
        parsed = json.load(handle)
    print(json.dumps(parsed, indent=4, sort_keys=True))

print_schema('train.avsc')


### 데이터세트 준비하기


Avro dataset API를 사용하여 `train.avro`를 TensorFlow 데이터세트로 로드합니다.


In [ ]:
features = {
    'features[*]': tfio.experimental.columnar.VarLenFeatureWithRank(dtype=tf.int32),
    'label': tf.io.FixedLenFeature(shape=[], dtype=tf.int32, default_value=-100),
    'dataType': tf.io.FixedLenFeature(shape=[], dtype=tf.string)
}

schema = tf.io.gfile.GFile('train.avsc').read()

dataset = tfio.experimental.columnar.make_avro_record_dataset(file_pattern=['train.avro'],
                                                              reader_schema=schema,
                                                              features=features,
                                                              shuffle=False,
                                                              batch_size=3,
                                                              num_epochs=1)

for record in dataset:
    print(record['features[*]'])
    print(record['label'])
    print(record['dataType'])
    print("--------------------")


위의 예는 `train.avro`를 tensorflow 데이터세트로 변환합니다. 데이터세트의 각 요소는 키가 기능 이름이고 값이 변환된 희소 또는 밀집 텐서인 사전입니다. 예를 들어, `features`, `label`, `dataType` 필드를 각각 VarLenFeature(SparseTensor), FixedLenFeature(DenseTensor) 및 FixedLenFeature(DenseTensor)로 변환합니다. batch_size가 3이므로 `train.avro`의 3개 레코드가 결과 데이터세트에서 하나의 요소로 강제 변환됩니다. 레이블이 null인 `train.avro`의 첫 번째 레코드에 대해 avro reader는 이를 지정된 기본값(-100)으로 바꿉니다. 이 예에서 `train.avro`에는 총 4개의 레코드가 있습니다. 배치 크기가 3이므로 결과 데이터세트에는 3개의 요소가 포함되며 마지막 배치 크기는 1입니다. 그러나 사용자는 `drop_final_batch`를 활성화하여 크기가 배치 크기보다 작은 경우 마지막 배치를 삭제할 수도 있습니다. 예를 들면 다음과 같습니다.


In [ ]:
dataset = tfio.experimental.columnar.make_avro_record_dataset(file_pattern=['train.avro'],
                                                              reader_schema=schema,
                                                              features=features,
                                                              shuffle=False,
                                                              batch_size=3,
                                                              drop_final_batch=True,
                                                              num_epochs=1)

for record in dataset:
    print(record)


또한 num_parallel_reads를 늘려 avro 구문 분석/읽기 병렬 처리를 높임으로써 Avro 데이터를 원활하게 처리할 수 있습니다.


In [ ]:
dataset = tfio.experimental.columnar.make_avro_record_dataset(file_pattern=['train.avro'],
                                                              reader_schema=schema,
                                                              features=features,
                                                              shuffle=False,
                                                              num_parallel_reads=16,
                                                              batch_size=3,
                                                              drop_final_batch=True,
                                                              num_epochs=1)

for record in dataset:
    print(record)


`make_avro_record_dataset`의 자세한 사용법은 <a target="_blank" href="https://www.tensorflow.org/io/api_docs/python/tfio/experimental/columnar/make_avro_record_dataset">API 문서</a>를 참조하세요.


### Avro 데이터세트로 tf.keras 모델 훈련하기

이제 mnist 데이터세트를 기반으로 하는 Avro 데이터세트를 사용하여 tf.keras 모델 훈련의 엔드 투 엔드 예제를 살펴보겠습니다.


Avro dataset API를 사용하여 `train.avro`를 TensorFlow 데이터세트로 로드합니다.


In [ ]:
features = {
    'features[*]': tfio.experimental.columnar.VarLenFeatureWithRank(dtype=tf.int32),
    'label': tf.io.FixedLenFeature(shape=[], dtype=tf.int32, default_value=-100),
}


schema = tf.io.gfile.GFile('train.avsc').read()

dataset = tfio.experimental.columnar.make_avro_record_dataset(file_pattern=['train.avro'],
                                                              reader_schema=schema,
                                                              features=features,
                                                              shuffle=False,
                                                              batch_size=1,
                                                              num_epochs=1)


간단한 keras 모델 정의:


In [ ]:
def build_and_compile_cnn_model():
    model = tf.keras.Sequential()
    model.compile(optimizer='sgd', loss='mse')
    return model

model = build_and_compile_cnn_model()


### Avro 데이터세트를 사용하여 keras 모델 훈련하기


In [ ]:
def extract_label(feature):
  label = feature.pop('label')
  return tf.sparse.to_dense(feature['features[*]']), label

model.fit(x=dataset.map(extract_label), epochs=1, steps_per_epoch=1, verbose=1)


Avro 데이터세트는 레코드, 맵, 배열, 분기 및 열거의 레코드를 포함하여 모든 avro 데이터를 구문 분석하고 TensorFlow 텐서로 강제 변환할 수 있습니다. 구문 분석 정보는 키가 데이터 값을 구문 분석하는 방법을 인코딩하는 맵으로 데이터를 TensorFlow 텐서로 강제 변환하는 방법을 인코딩하는 맵으로 전달됩니다. 이것으로 기본 유형(예: bool, int, long, float, double, string)뿐만 아니라 텐서 유형(예: 희소 또는 밀집)도 결정됩니다. TensorFlow의 파서 유형 목록(표 1 참조)과 기본 유형의 강제 변환(표 2)이 제공됩니다.

표 1 지원되는 TensorFlow 파서 유형:

TensorFlow 파서 유형 | TensorFlow 텐서 | 설명
--- | --- | ---
tf.FixedLenFeature([], tf.int32) | 밀집 텐서 | 고정 길이 특성을 구문 분석합니다. 즉, 모든 행은 동일한 일정 요소 수를 갖습니다. 단 하나의 요소 또는 각 행에 대해 항상 동일한 수의 요소를 갖는 배열을 예로 들 수 있습니다.
tf.SparseFeature(index_key=['key_1st_index', 'key_2nd_index'], value_key='key_value', dtype=tf.int64, size=[20, 50]) | 희소 텐서 | 각 행에 인덱스 및 값의 가변 길이 목록이 있는 희소 특성을 구문 분석합니다. 'index_key'는 인덱스를 식별합니다. 'value_key'는 값을 식별합니다. 'dtype'은 데이터 유형입니다. 'size'는 각 인덱스 항목에 대해 예상되는 최대 인덱스 값입니다.
tfio.experimental.columnar.VarLenFeatureWithRank([],tf.int64) | 희소 텐서 | 가변 길이 특성을 구문 분석합니다. 이는 각 데이터 행이 가변적 수의 요소를 가질 수 있음을 의미합니다. 예를 들어, 첫 번째 행에는 5개의 요소가 있고 두 번째 행에는 7개의 요소가 있습니다.

표 2 Avro 유형에서 TensorFlow 유형으로 지원되는 변환:

Avro 기본 유형 | TensorFlow 기본 유형
--- | ---
boolean: 바이너리 값 | tf.bool
바이트: 부호 없는 8비트 바이트 시퀀스 | tf.string
double: 배정밀도 64비트 IEEE 부동 소수점 숫자 | tf.float64
enum: 열거 유형 | 기호 이름을 사용하는 tf.string
float: 단정밀도 32비트 IEEE 부동 소수점 숫자 | tf.float32
int: 부호 있는 32비트 정수 | tf.int32
long: 부호 있는 64비트 정수 | tf.int64
null: 값 없음 | 기본값 사용
string: 유니코드 문자 시퀀스 | tf.string


Avro dataset API의 전체 예제는 <a target="_blank" href="https://github.com/tensorflow/io/blob/master/tests/test_parse_avro.py#L437">테스트</a> 내에서 제공됩니다.
